In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
#sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
#sys.path.insert(0,'/home/ikwak2/hmd/iy_classifier')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
from models import *
from Generator0 import *

In [2]:
root_dir = 'physionet.org/files/circor-heart-sound/1.0.3'
training_data_file = root_dir + '/' + 'training_data.csv'
training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [3]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


Fri Jul  8 17:16:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   35C    P8    14W /

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

In [5]:
data_folder =  'physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/home/ubuntu/data/hmd/murmur/train'
test_folder = '/home/ubuntu/data/hmd/murmur/test'


In [6]:
model_folder = 'tmp_model4'

## Modeling

test 모형이 get_toy4_1, get_toy4_2 라고 하자

In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [8]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

In [9]:
model1 = get_toy4_1((100, 313, 1))
model2 = get_toy4_2((100, 313, 1))


In [10]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mel (InputLayer)                [(None, 100, 313, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 98, 311, 16)  160         mel[0][0]                        
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 49, 155, 16)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 45, 151, 32)  12832       max_pooling2d[0][0]              
______________________________________________________________________________________________

In [11]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mel (InputLayer)                [(None, 100, 313, 1) 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 98, 311, 16)  160         mel[0][0]                        
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 49, 155, 16)  0           conv2d_4[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 45, 151, 32)  12832       max_pooling2d_4[0][0]            
____________________________________________________________________________________________

In [12]:
from tensorflow.keras.utils import plot_model
plot_model(model1, to_file='model.png')

AttributeError: 'ListWrapper' object has no attribute 'name'

In [13]:
plot_model(model2, to_file='model.png')

AttributeError: 'ListWrapper' object has no attribute 'name'

In [14]:
#patient_files = find_patient_files(data_folder)
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

#num_patient_files = len(patient_files)

In [15]:
features_trn = get_features_3lb(train_folder, patient_files_trn)
features_test = get_features_3lb(test_folder, patient_files_test)

In [16]:
n_epoch = 2
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn[0]['age'],features_trn[0]['sex'], features_trn[0]['hw'], features_trn[0]['preg'], features_trn[0]['loc'], 
           features_trn[0]['mel1']], 
                        features_trn[1],  ## our Y
                        **params)()

ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
           features_test[0]['mel1']], 
                        features_test[1],  ## our Y
                        **params_no_shuffle)()

    
model1.fit(TrainDGen_1,
          validation_data = ([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], 
                              features_test[0]['preg'], features_test[0]['loc'], features_test[0]['mel1']], 
                             features_test[1]), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(features_trn[1])/64),
                             epochs = n_epoch)

Epoch 1/2
40/40 [==============================] - 19s 470ms/step - loss: 0.7530 - accuracy: 0.7523 - val_loss: 0.6148 - val_accuracy: 0.7448 - lr: 9.9851e-04
Epoch 2/2
40/40 [==============================] - 19s 465ms/step - loss: 0.5834 - accuracy: 0.7648 - val_loss: 0.5852 - val_accuracy: 0.7448 - lr: 5.0500e-04


In [17]:
n_epoch = 2
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn[0]['age'],features_trn[0]['sex'], features_trn[0]['hw'], features_trn[0]['preg'], features_trn[0]['loc'], 
           features_trn[0]['mel1']], 
                        features_trn[2],  ## our Y
                        **params)()

ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
           features_test[0]['mel1']], 
                        features_test[2],  ## our Y
                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], 
                              features_test[0]['preg'], features_test[0]['loc'], features_test[0]['mel1']], 
                             features_test[2]), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(features_trn[1])/64),
                             epochs = n_epoch)

Epoch 1/2
40/40 [==============================] - 19s 466ms/step - loss: 0.8496 - accuracy: 0.5125 - val_loss: 0.6803 - val_accuracy: 0.5626 - lr: 9.9851e-04
Epoch 2/2
40/40 [==============================] - 19s 463ms/step - loss: 0.6918 - accuracy: 0.5320 - val_loss: 0.6778 - val_accuracy: 0.5705 - lr: 5.0500e-04


### 팀코드 수정해야 할 함수들..

In [18]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, mel_shape = (100, 313, 1)) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    d = {'model1': m_name1, 'model2': m_name2, 'mel_shape': mel_shape, 'model_fnm1': filename1, 'model_fnm2': filename2}    
    with open(info_fnm, 'wb') as f:
        pk.dump(d, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [19]:
model_folder

'tmp_model4'

In [20]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'toy1', m_name2 = 'toy2')

1

In [21]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

In [22]:
def run_challenge_model(model, data, recordings, verbose):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    
    if model['model1'] == 'toy1' :
        model1 = get_toy4_1(model['mel_shape'])
    if model['model2'] == 'toy2' :
        model2 = get_toy4_2(model['mel_shape'])
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
    
#    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    features['mel1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_melspec(recordings[i]/ 32768)[0]
        features['mel1'].append(mel1)

    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
        
    features['mel1'] = np.array(features['mel1'])
#    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1']])

    # Get classifier probabilities.
    idx1 = res1.argmax(axis=0)[0]
    murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
    outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
#    idx = np.argmax(prob1)

    ## 이부분도 생각 필요.. rule 을 cost를 maximize 하는 기준으로 threshold 탐색 필요할지도..
    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities

### 대회 평가용 run_model 함수 (수정 불필요)

In [23]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [24]:
## 우리 모형 저장된 폴더이름
model_folder

'tmp_model4'

In [25]:
## 테스트 데이터 폴더
test_folder

'/home/ubuntu/data/hmd/murmur/test'

In [26]:
## 테스트 데이터에 모형 돌려서 스코어 저장할 폴더
output_folder = '/home/ubuntu/hmd/notebooks/out2'

In [27]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 1)

Loading Challenge model...
Running model on Challenge data...
Done.


In [28]:
!cat /home/ubuntu/hmd/notebooks/out2/49628.csv

#49628
Present,Unknown,Absent,Abnormal,Normal
0,0,1,1,0
0.37117127,0.012293197,0.6165355,0.54467726,0.45532268


수정완료 기록 잘 됨

In [29]:
from evaluate_model import *

In [30]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.622,0.424,0.281,0.728,0.375,25689.450

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.722,0.707,0.530,0.586,0.350,20898.997

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.654,0.500,0.712
AUPRC,0.326,0.076,0.871
F-measure,0.000,0.000,0.842
Accuracy,0.000,0.000,1.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.722,0.722
AUPRC,0.727,0.687
F-measure,0.368,0.693
Accuracy,0.235,0.957



epoch 2 트레이닝 해서 결과는 별로.. 기록은 잘 됨